In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder.getOrCreate()

23/02/02 20:19:42 WARN Utils: Your hostname, Lisbeth resolves to a loopback address: 127.0.1.1; using 192.168.100.49 instead (on interface wlp0s20f3)
23/02/02 20:19:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/02 20:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
emp = (
    spark.read.csv("EMPRESAS_CNPJ.csv", sep=';')
    .toDF(
        'cnpj_basico',
        'razao_social',
        'natureza_juridica',
        'qualificacao_responsavel',
        'capital_social',
        'porte',
        'ente_responsavel')
)

est = (
    spark.read.csv("ESTABELE_CNPJ.csv", sep=';')
    .toDF(
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'id_matriz',
        'nome_fantasia',
        'situacao',
        'situacao_data',
        'situacao_motivo',
        'cidade_exterior',
        "pais_codigo",
        'inicio_data',
        'cnae_principal',
        'cnae_secundario',
        'logradouro_tipo',
        'logradouro',
        'lograoduro_numero',
        'logradouro_complemento',
        'bairro',
        'cep',
        'uf',
        'municipio',
        'ddd1',
        'telefone1',
        'ddd2',
        'telefone2',
        'ddd_fax',
        'fax',
        'email',
        'situacao_especial',
        'situacao_especial_data')
)

In [4]:
emp = (
    emp
    .select(
        'cnpj_basico',
        'natureza_juridica',
        'qualificacao_responsavel',
        'capital_social',
        'porte',
    )
)

est = (
    est
    .select(
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'id_matriz',
        'situacao',
        'situacao_data',
        'situacao_motivo',
        'inicio_data',
        'cnae_principal',
        'cep',
        'uf',
    )
)

In [5]:
est = est.filter(f.col('uf') == 'MG')

In [6]:
est = (
    est
    .join(emp, on='cnpj_basico', how='inner')
)

### Formatando as datas

In [7]:
est = (
    est
    .withColumn('situacao_data', f.to_date(f.col('situacao_data').cast('string'), "yyyyMMdd"))
    .withColumn('inicio_data', f.to_date(f.col('inicio_data').cast('string'), "yyyyMMdd"))
)

In [8]:
est = (
    est
    .withColumn('cnpj', f.concat_ws("",
                f.col('cnpj_basico'), f.col('cnpj_ordem'), f.col('cnpj_dv')
                ))
    .drop('cnpj_basico', 'cnpj_dv', 'cnpj_ordem')
)

In [9]:
est.groupBy('natureza_juridica').count().show(500)

23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/02 20:20:43 WARN RowBasedKeyValueBatch: Calling spill() on

+-----------------+-------+
|natureza_juridica|  count|
+-----------------+-------+
|             1104|    596|
|             3280|     21|
|             1112|    189|
|             2224|    839|
|             3131|   4974|
|             2305|  20807|
|             2100|    169|
|             3999| 164264|
|             1074|     28|
|             2046|   7714|
|             3328|     45|
|             2097|     26|
|             1082|     31|
|             5029|     14|
|             8885|    318|
|             4014|  18239|
|             1139|     69|
|             3115|     35|
|             2062|1520666|
|             2267|      3|
|             2127|   4328|
|             3301|    313|
|             1023|    921|
|             2178|     60|
|             1147|    147|
|             2283|     52|
|             2038|   3239|
|             2321|   7094|
|             1210|    290|
|             1066|    876|
|             2348|    125|
|             3069|   2743|
|             3220| 